In [1]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split

In [127]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model, load_model
from keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, LSTM, GlobalMaxPooling1D, Dropout 

### **Read Dataset**

In [3]:
df = pd.read_excel('data_train_intent_detection.xlsx')

In [4]:
df.head()

,مشکل غیرفعالسازی آوای انتظار,مشکل فعالسازی آوای انتظار,مشکل تغییر آوای انتظار,مشکل پخش آوای انتظار,لیست آواهای انتظار,دریافت کد آوای انتظار خط خود,دریافت کد آوای انتظار خط دیگران,روش فعالسازی آوای انتظار,روش غیرفعالسازی آوای انتظار,روش تغییرآوای انتظار,خارج از دامنه
0,از طریق 8989 امکان لغو اوای انتظار وجود ندارد...,خیلی تلاش کردم اوای انتظار رو فعال کنم اما فع...,سلام علیکم حاج خانوم خوبی من ببخشید مزاحمتون م...,سلام احوال شما خوبه؟ من یه نوحه انتخاب کردم وا...,همه کدای اوای همراه اولو از کجا بگیرم و یکیشو...,یه اهنگ گذاشتم رو خطم نمیدونم کدش چیه واسه دو...,سلام علیکم ما یه نوحه انتخاب کرده بودیم که با ...,الان باهاتون داشتم صحبت میکردم قطع شد من میخوا...,سلام من چطوری میتونم پیشواز خط بابامو بردارم ا...,سلام سال نو مبارک میخواستم ببینم میشه با کدی ک...,سلام من آوای انتظاری که انتخاب کردم تو سایت که...
1,با پیام به 8989 اقدام به غیرفعالسازی اوای انت...,از کجا پیگیری کنم اوای انتظارم فعال نمیشه,سلام حال شما خوب هستین ببخشید من یه پیشوازی بر...,سلام وقتتون بخیر خانومم سه روز پیش یه آوای انت...,همه کدای اوای همراه اولو چجوری بگیرم,یه اهنگ رو خطم گذاشتم میخوام کدشو بهم بدید وا...,سلام من این آهنگ ای دل اگر عاشقی در پی دلدار ب...,سلام من برای فعالسازی خطم وارد سایت همراه من ش...,سلام حال شما خوبه؟من یه چندماهی میشه آوای انتظ...,سلام من خطم بخاطر بدهی قطع شده الان میخوام پیش...,سلام چرا انقدر این پیشواز میزاریم صداش بی‌کیفی...
2,برای غیرفعالسازی اوای انتظار پیام به 8989 داد...,خیلی تلاش کردم اوای انتظار رو فعال کنم اما او...,سلام میکنم خدمتتون خانم عرض ادب غرض از مزاحمت ...,سلام من سه چهار روزه برای خط دخترم پیشواز فعال...,همه کد اواهای انتظار همراه اولو چطور دریافت کنم,دوستم یه اهنگ پیشواز رو خط همراه اولش گذاشته ...,سلام من به رفیقم زنگ زدم بعد نوحه حسین حسین آ...,سلام این کد دستوری همراه اول برای فعالسازی همش...,سلام ببخشید من میخوام آوای انتظار خط همسرمو غی...,سلام خوب هستید؟من میخواستم آوای انتظار گوشی ما...,وقتتون بخیر باشه این آوای انتظاری که من انتخاب...
3,از طریق ارسال پیام به 8989 اقدام به غیرفعالسا...,برای رفع مشکل فعالسازی پیشواز از کجا باید پیگ...,سلام حال شما خوبه این دوست من خیلی به یه خوانن...,سلام خوبید؟من آوای انتظارم اولش میخونه بعدش بو...,کدای پیشواز همشو از کجا بگیرم,پیدا کردن اوام به چه ترتیبیه,سلام من یه چند هفته پیش پیشواز گذاشتم روی خط م...,سلام من چطوری میتونم واسه دوتا از خطهام همزمان...,سلام صبح بخیر،گوشی داداشم خراب شده الان میخواد...,سلام از صبح چند نفر بهم زنگ زدن گفتن چقدر آوای...,سلام من پیشواز از آهنگ طلیسچی گذاشتم صداش خیلی...
4,با ارسال 5 به 8989 میخواستم اوای انتظار غیرفع...,از هر راهی اقدام می کنم اوای انتظار فعال نمیش...,علو خوبین سلام عرض میکنم. عیدتون مبارک. طاعاتت...,سلام من آوای انتظار فعال کردم واسه خطم اما وقت...,همه کدای اوای همراه اول رو از کجا بگیرم و یکی...,رفیق کد انتظارمو ازم خواست یادم نبود چیه,سلام خوبید؟من با همکارم تماس گرفتم بعد آوای ان...,سلام میخواستم بپرسم که برای فعال کردن پیشواز ب...,سلام من تازه پیشواز گذاشتم اما خوشم نیومد میتو...,سلام من آوای انتظارمو از محرم تغییر ندادم همه ...,سلام


In [5]:
df.shape

(55, 11)

In [6]:
df.iloc[0]

مشکل غیرفعالسازی آوای انتظار        از طریق 8989 امکان لغو اوای انتظار وجود ندارد...
مشکل فعالسازی آوای انتظار           خیلی تلاش کردم اوای انتظار رو فعال کنم اما فع...
مشکل تغییر آوای انتظار             سلام علیکم حاج خانوم خوبی من ببخشید مزاحمتون م...
مشکل پخش آوای انتظار               سلام احوال شما خوبه؟ من یه نوحه انتخاب کردم وا...
لیست آواهای انتظار                  همه کدای اوای همراه اولو از کجا بگیرم و یکیشو...
دریافت کد آوای انتظار خط خود        یه اهنگ گذاشتم رو خطم نمیدونم کدش چیه واسه دو...
دریافت کد آوای انتظار خط دیگران    سلام علیکم ما یه نوحه انتخاب کرده بودیم که با ...
روش فعالسازی آوای انتظار           الان باهاتون داشتم صحبت میکردم قطع شد من میخوا...
روش غیرفعالسازی آوای انتظار        سلام من چطوری میتونم پیشواز خط بابامو بردارم ا...
روش تغییرآوای انتظار               سلام سال نو مبارک میخواستم ببینم میشه با کدی ک...
خارج از دامنه                      سلام من آوای انتظاری که انتخاب کردم تو سایت که...
Name: 0, dtype: object

In [7]:
list(df.iloc[0].keys())

['مشکل غیرفعالسازی آوای انتظار',
 'مشکل فعالسازی آوای انتظار',
 'مشکل تغییر آوای انتظار',
 'مشکل پخش آوای انتظار',
 'لیست آواهای انتظار',
 'دریافت کد آوای انتظار خط خود',
 'دریافت کد آوای انتظار خط دیگران',
 'روش فعالسازی آوای انتظار',
 'روش غیرفعالسازی آوای انتظار',
 'روش تغییرآوای انتظار',
 'خارج از دامنه']

In [8]:
def create_dataset(df):
  output = {
        "text": [],
        "category": []
    }

  for data in df.iloc:
    for i,d in enumerate(data):
      output["text"].append(d)
      output["category"].append(list(df.iloc[0].keys())[i])
   
  return output

In [9]:
data = create_dataset(df)

In [10]:
print(len(data["text"]))
print(len(data["category"]))

605
605


In [11]:
print(data["text"][0])
print(data["category"][0])

 از طریق 8989 امکان لغو اوای انتظار وجود ندارد از چه روشی می توانم پیگیری کنم
مشکل غیرفعالسازی آوای انتظار


In [12]:
dataset = pd.DataFrame(data)

In [13]:
dataset.head()

,text,category
0,از طریق 8989 امکان لغو اوای انتظار وجود ندارد...,مشکل غیرفعالسازی آوای انتظار
1,خیلی تلاش کردم اوای انتظار رو فعال کنم اما فع...,مشکل فعالسازی آوای انتظار
2,سلام علیکم حاج خانوم خوبی من ببخشید مزاحمتون م...,مشکل تغییر آوای انتظار
3,سلام احوال شما خوبه؟ من یه نوحه انتخاب کردم وا...,مشکل پخش آوای انتظار
4,همه کدای اوای همراه اولو از کجا بگیرم و یکیشو...,لیست آواهای انتظار


In [14]:
dataset.shape

(605, 2)

In [15]:
dataset['category'].value_counts()

مشکل غیرفعالسازی آوای انتظار       55
مشکل فعالسازی آوای انتظار          55
مشکل تغییر آوای انتظار             55
مشکل پخش آوای انتظار               55
لیست آواهای انتظار                 55
دریافت کد آوای انتظار خط خود       55
دریافت کد آوای انتظار خط دیگران    55
روش فعالسازی آوای انتظار           55
روش غیرفعالسازی آوای انتظار        55
روش تغییرآوای انتظار               55
خارج از دامنه                      55
Name: category, dtype: int64

In [16]:
#delete NaN
dataset = dataset.dropna()

In [17]:
dataset.shape

(498, 2)

**stopword**

In [30]:
def read_stop_words(filename):
  with open(filename) as stopwords_file:
    stopwords = stopwords_file.readlines()
  stopwords = [line.replace('\n', '') for line in stopwords] 
  return stopwords

In [32]:
stopwords = read_stop_words('stopwords.txt')
print(len(stopwords))

1316


In [18]:
# install hazm library
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 55.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 29.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=56b2ff4d381f9db2eb3247f90f6edd58dccb2bfeb40547f94a18b3b931ee477c
  Stored in directory: /root/.cache/pip/wheels/ac/62/f6/88933dadcd64a1614894614aa68cf57c4b8e5256acb650b1f1
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp39-cp39-linux_x86_64.whl size=180371 sha256=7ca9595499c9f8d1ebb65a7a88c9186489f6f5a2bfb38d7d99740842b244f87a
  Stored in directory: /root/.cache/pip/wheels/8e/ff/82/9326b96f96f47472e02c453697b225813e4650c0ed4df2cd49
Successfully built nltk libwapiti
  Attempting unin

In [19]:
from hazm import word_tokenize

In [20]:
text = dataset['text'].values

In [21]:
text[0]

' از طریق 8989 امکان لغو اوای انتظار وجود ندارد از چه روشی می توانم پیگیری کنم'

In [22]:
text_preprocessing = [re.sub('/s', '', line) for line in text]

In [33]:
word_tokenized = [word_tokenize(t) for t in text]
word_tokenized_filtered = [[w for w in sentence if w not in stopwords] for sentence in word_tokenized]

In [36]:
print(word_tokenized[0])

['از', 'طریق', '8989', 'امکان', 'لغو', 'اوای', 'انتظار', 'وجود', 'ندارد', 'از', 'چه', 'روشی', 'می', 'توانم', 'پیگیری', 'کنم']


In [35]:
print(word_tokenized_filtered[0])

['8989', 'امکان', 'لغو', 'اوای', 'انتظار', 'روشی', 'پیگیری']


In [95]:
def concat_words_of_sentence(word_tokenized_filtered):
  corpus = []
  for sentence in word_tokenized_filtered:
    s = ' '
    for w in sentence:
      s += w + ' '
    corpus.append(s)  
  return corpus

In [96]:
corpus = concat_words_of_sentence(word_tokenized_filtered)

In [120]:
print(corpus[3])

 سلام احوال خوبه نوحه انتخاب واسه پیشوازم پخش نمیشه خطم بوق ساده میخوره چیکار 


### **one hot Y**

In [99]:
Y_one_hot = pd.get_dummies(dataset['category'])

In [100]:
category_types = Y_one_hot.columns.values

In [101]:
category_types

array(['خارج از دامنه', 'دریافت کد آوای انتظار خط خود',
       'دریافت کد آوای انتظار خط دیگران', 'روش تغییرآوای انتظار',
       'روش غیرفعالسازی آوای انتظار', 'روش فعالسازی آوای انتظار',
       'لیست آواهای انتظار', 'مشکل تغییر آوای انتظار',
       'مشکل غیرفعالسازی آوای انتظار', 'مشکل فعالسازی آوای انتظار',
       'مشکل پخش آوای انتظار'], dtype=object)

In [102]:
Y = pd.get_dummies(dataset['category']).values

In [103]:
Y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], dtype=uint8)

### **Text Tokenizer**

In [104]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [105]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

In [106]:
vocab_size = len(tokenizer.word_index)+1

In [107]:
vocab_size

1087

In [108]:
data_to_sequences = tokenizer.texts_to_sequences(corpus)
maxlen = max_length(data_to_sequences)

In [109]:
maxlen

29

In [110]:
pad_data_to_sequences = pad_sequences(data_to_sequences, maxlen=maxlen)

In [121]:
X_train, X_test, y_train, y_test = train_test_split(pad_data_to_sequences, Y, shuffle=True, test_size=0.3, random_state=42)

In [122]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(348, 29)
(150, 29)
(348, 11)
(150, 11)


### **Train The Model**

In [128]:
embedding_dim = 50

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(Bidirectional(LSTM(200, return_sequences=True)))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.3))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(category_types), activation='softmax'))

In [129]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [125]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 29, 50)            54350     
                                                                 
 bidirectional_4 (Bidirectio  (None, 29, 400)          401600    
 nal)                                                            
                                                                 
 global_max_pooling1d_4 (Glo  (None, 400)              0         
 balMaxPooling1D)                                                
                                                                 
 dropout_8 (Dropout)         (None, 400)               0         
                                                                 
 dense_8 (Dense)             (None, 200)               80200     
                                                                 
 dropout_9 (Dropout)         (None, 200)              

In [126]:
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=256)

Epoch 1/50
2/2 [==============================] - 5s 879ms/step - loss: 2.3967 - accuracy: 0.1034 - val_loss: 2.3954 - val_accuracy: 0.1333
Epoch 2/50
2/2 [==============================] - 0s 322ms/step - loss: 2.3899 - accuracy: 0.1178 - val_loss: 2.3943 - val_accuracy: 0.0867
Epoch 3/50
2/2 [==============================] - 0s 216ms/step - loss: 2.3837 - accuracy: 0.1236 - val_loss: 2.3936 - val_accuracy: 0.0867
Epoch 4/50
2/2 [==============================] - 1s 327ms/step - loss: 2.3724 - accuracy: 0.1437 - val_loss: 2.3946 - val_accuracy: 0.0867
Epoch 5/50
2/2 [==============================] - 1s 304ms/step - loss: 2.3610 - accuracy: 0.1408 - val_loss: 2.4042 - val_accuracy: 0.0933
Epoch 6/50
2/2 [==============================] - 0s 322ms/step - loss: 2.3596 - accuracy: 0.1264 - val_loss: 2.4102 - val_accuracy: 0.1333
Epoch 7/50
2/2 [==============================] - 0s 253ms/step - loss: 2.3372 - accuracy: 0.1379 - val_loss: 2.3922 - val_accuracy: 0.1000
Epoch 8/50
2/2 [====

In [117]:
model.save('model.h5')